<a href="https://colab.research.google.com/github/sagiodev/stablediffusion_webui/blob/master/LoRA_trainer_kohya_ss_SDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lora Trainer (kohya_ss GUI)
### [Guide to use this notebook](https://stable-diffusion-art.com/train-lora/) - Leave comment if you have questions.

Latest version of this notebook can be found [here](https://colab.research.google.com/github/sagiodev/SDA-lora-training/blob/main/LoRA_trainer_kohya_ss_SDA.ipynb).

This notebook launches [kohya_ss GUI](https://github.com/bmaltais/kohya_ss/tree/ed4e3b0239a40506de9a17e550e6cf2d0b867a4f).


## Instruction
1. Review the info on the first cell.
2. Run first cell to upload images. Some paths will be printed.
3. Run the second cell to launch the LoRA trainer. You will need the paths from the first cell.

## Log
- 12/27/2023: Updated diffuser version
- 11/16/2023: Allow special characters in password
- 10/31/2023: Use CUDA 11.7 when compiling bitsandbytes


In [ ]:
#@title <font size="5" color="orange">Step 1: Upload images and print folder names </font>
#@markdown Begineers: Use a different `Project_folder` each time when you upload the images.
from google.colab import drive
drive.mount('/content/drive')

Project_folder = 'AI_PICS/training/AndyLau' #@param {type:"string"}
dataset_name = 'AndyLau' #@param {type:"string"}
Number_of_epoches = 100 #@param {type:"integer"}
Lora_output_path = 'AI_PICS/Lora' #@param {type:"string"}

# construct paths
projectPath = '/content/drive/MyDrive/' + Project_folder
imagePath = projectPath + '/' + str(Number_of_epoches) + '_'+ dataset_name
loraPath = '/content/drive/MyDrive/' + Lora_output_path

!mkdir -p {loraPath}

import os
from google.colab import files
import shutil

if os.path.exists(imagePath):
  print(f'Error: Folder {imagePath} already exists. Please use a different project folder or dataset names. Skip uploading.\n')
else:
  !mkdir -p {imagePath}
  uploaded = files.upload()
  for filename in uploaded.keys():
      dst_path = imagePath + '/' + filename
      shutil.move(filename, dst_path)
  print('Images uploaded successfully.\n')

# print image paths
print(f"Image folder to caption: {imagePath}")
print(f"Lora Image folder: {projectPath}")
print(f"Lora output folder: {loraPath}")



In [ ]:
#@title <font size="5" color="orange">Step 2: Train LoRA </font>
#@markdown Select username and password to prevent unauthorized access.
username = 'a' #@param {type:"string"}
password = 'a' #@param {type:"string"}
Clear_Log = True #@param {type:"boolean"}

def clear():
    from IPython.display import clear_output; return clear_output()

from google.colab import drive
drive.mount('/content/drive')


!pip install dadaptation==3.1 diffusers[torch]==0.24.0 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1 huggingface-hub==0.19.4
!pip install lion-pytorch==0.0.6 lycoris_lora==1.8.0.dev6 open-clip-torch==2.20.0 prodigyopt==1.0 pytorch-lightning==1.9.0 safetensors==0.3.1 timm==0.6.12
!pip install tk==0.1.0 transformers==4.30.2 voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 omegaconf


# Install bitsandbytes
!git clone -b 0.41.0 https://github.com/TimDettmers/bitsandbytes
%cd /content/bitsandbytes
!CUDA_VERSION=117 make cuda11x
!python setup.py install

%cd /content
!git clone https://github.com/bmaltais/kohya_ss.git
#!git clone https://github.com/camenduru/kohya_ss.git
%cd kohya_ss/
!git checkout v21.8.9

if Clear_Log:
  clear()

# add pwd to python path or else blip captioning won't work
%env PYTHONPATH=/env/python:/content/kohya_ss
!python kohya_gui.py --headless --share --username='{username}' --password='{password}'

